In [0]:
%run ./_setup

In [0]:
from brickbyte import BrickByte

bb = BrickByte(
    sources=["source-faker"],
    destination="destination-databricks",
    destination_install="git+https://github.com/park-peter/brickbyte.git#subdirectory=integrations/destination-databricks-py"
)
bb.setup()

In [0]:
import airbyte as ab

FORCE_FULL_REFRESH = True
cache = bb.get_or_create_cache()

# Configure the Faker source (generates fake data for testing)
source = ab.get_source(
    "source-faker",
    config={"count": 100},
    local_executable=bb.get_source_exec_path("source-faker")
)
source.check()
source.select_all_streams()

In [0]:
# Configure the Databricks destination - fill in your connection details
destination = ab.get_destination(
    "destination-databricks",
    config={
        "server_hostname": "",  # e.g., "adb-xxx.azuredatabricks.net"
        "http_path": "",        # e.g., "/sql/1.0/warehouses/abc123"
        "token": "",            # Your Databricks PAT
        "catalog": "",          # Unity Catalog name
        "schema": "",           # Target schema
    },
    local_executable=bb.get_destination_exec_path()
)

write_result = destination.write(source, cache=cache, force_full_refresh=FORCE_FULL_REFRESH)


In [0]:
bb.cleanup()